In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [23]:
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
num_heads =4

In [3]:
device

'cpu'

In [4]:
torch.manual_seed(1337)

In [5]:
import os

In [6]:
os.getcwd()

'/content'

In [7]:
os.chdir("/content/drive/MyDrive/ng-video-lecture-master")

In [8]:
with open('input.txt', "r", encoding="utf-8") as f:
  text = f.read()

In [9]:
chars = sorted(list(set(text)))

In [10]:
vocab_size = len(chars)
vocab_size

65

In [11]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

In [12]:
encode = lambda s : [stoi.get(c) for c in s ]
decode = lambda l : "".join(itos.get(i) for i in l)

In [13]:
data = torch.tensor(encode(text), dtype=torch.long)

In [14]:
n = int (0.9*len(data))

In [15]:
train_data=data[:n]
val_data = data[n:]

In [16]:
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42])

In [17]:
train_data[1:block_size+1]

tensor([47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44, 53,
        56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1])

In [18]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for ch in range(block_size):
  context =  x[:ch+1]
  target = y[ch]
  print(f"context : {context}, target:{target}")

context : tensor([18]), target:47
context : tensor([18, 47]), target:56
context : tensor([18, 47, 56]), target:57
context : tensor([18, 47, 56, 57]), target:58
context : tensor([18, 47, 56, 57, 58]), target:1
context : tensor([18, 47, 56, 57, 58,  1]), target:15
context : tensor([18, 47, 56, 57, 58,  1, 15]), target:47
context : tensor([18, 47, 56, 57, 58,  1, 15, 47]), target:58
context : tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]), target:47
context : tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]), target:64
context : tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]), target:43
context : tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43]), target:52
context : tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]), target:10
context : tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10]), target:0
context : tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0]), target:14
context : tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10

In [19]:
def get_batches(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y = x.to(device), y.to(device)
  return x,y

In [20]:
def estimate_loss():
  out = {}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      xb,yb = get_batches(split)
      logits, loss = m(xb, yb)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [21]:
b,t = xb.shape
xb.view(b*t).shape

NameError: name 'xb' is not defined

In [22]:
print(xb)

NameError: name 'xb' is not defined

In [ ]:
class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None ):
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits , loss


  def generate(self, idx, max_new_tokens):
    for k in range(max_new_tokens):
      logits, loss = self(idx)
      if k == 0 :
        print(logits.shape)
      logits= logits[:,-1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)

    return idx



In [ ]:
idx = torch.zeros((1,1), dtype=torch.long , device=device)
model = BigramLanguageModel(vocab_size)
m = model.to(device)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(loss.item())


print(decode(m.generate(idx=idx, max_new_tokens = 100)[0].tolist()))


torch.Size([512, 65])
tensor(4.6057, grad_fn=<NllLossBackward0>)
4.605657577514648
torch.Size([1, 1, 65])

DtQOLwf'RIPD'Hmhs xvLwxat:sbYZwLEHy'pryOqOZm$!s$zheQ,;eMkjm,CQufzLw!iWn.e!zDGxA PsS3zdKY!KzGCeJydKJS


In [ ]:
idx = torch.zeros((2,2), dtype=torch.long , device=device)
print(decode(m.generate(idx=idx, max_new_tokens = 100)[0].tolist()))
print(decode(m.generate(idx=idx, max_new_tokens = 100)[1].tolist()))

torch.Size([2, 2, 65])


rFFJSak!g!wWSa.WxbjBiDeQ:scqpqvWrxb-du$WDaJWuBr-g-sOZKYwPifPV&& PDzluBatz$LDt:aRyU:aZfBUvfp:ZQsbZhli
torch.Size([2, 2, 65])


C;tBpMVAMUuAzduxvuh3DtB
CrKqoiGBx;nVTAkdxJKZmXi-$A.c!Z3zhMx-EsVLYG,X mWQI
kiwBv fnIaYyVV;apjoLEm-fRx


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

NameError: name 'model' is not defined

In [ ]:
import tqdm

In [ ]:
def estimate_loss():
  out = {}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      xb,yb = get_batches(split)
      logits, loss = m(xb, yb)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [ ]:
for iter in range(max_iters):
      # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batches("train")
  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())

step 0: train loss 4.6097, val loss 4.6070
step 100: train loss 4.4848, val loss 4.4847
step 200: train loss 4.3690, val loss 4.3658
step 300: train loss 4.2497, val loss 4.2539
step 400: train loss 4.1448, val loss 4.1477
step 500: train loss 4.0380, val loss 4.0378
step 600: train loss 3.9405, val loss 3.9473
step 700: train loss 3.8495, val loss 3.8498
step 800: train loss 3.7536, val loss 3.7556
step 900: train loss 3.6671, val loss 3.6765
step 1000: train loss 3.5890, val loss 3.5951
step 1100: train loss 3.5185, val loss 3.5208
step 1200: train loss 3.4463, val loss 3.4531
step 1300: train loss 3.3796, val loss 3.3832
step 1400: train loss 3.3113, val loss 3.3224
step 1500: train loss 3.2539, val loss 3.2572
step 1600: train loss 3.2028, val loss 3.2102
step 1700: train loss 3.1493, val loss 3.1562
step 1800: train loss 3.1044, val loss 3.1154
step 1900: train loss 3.0563, val loss 3.0636
step 2000: train loss 3.0186, val loss 3.0271
step 2100: train loss 2.9771, val loss 2.9832


In [ ]:
print(decode(m.generate(idx=idx, max_new_tokens = 100)[0].tolist()))

torch.Size([2, 2, 65])


Bus beis, w.
Fonirchenomastone heavare I ope t. kirugh?
Yvebreforiveenge nesed fes  PSLAEcuan3a? LEn


In [ ]:
B,T,C = 4,8,32
x = torch.randn(B,T,C)
head_size = 16

query = nn.Linear(C,head_size, bias= False)
key = nn.Linear(C,head_size, bias= False)
value = nn.Linear(C,head_size, bias= False)

q = query(x) #(B,T,head_size)
k = key(x) #(B,T,head_size)
v = value(x) #(B,T,head_size)

wei = q @ k.transpose(-2,-1) # (B,head_size,T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print(wei[0])
print(wei.shape)

out = wei @ v

print(out[0])
print(out.shape)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6119, 0.3881, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0711, 0.1138, 0.8150, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3576, 0.0473, 0.1256, 0.4695, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0063, 0.2960, 0.0177, 0.1327, 0.5473, 0.0000, 0.0000, 0.0000],
        [0.0365, 0.1685, 0.3274, 0.1337, 0.0643, 0.2696, 0.0000, 0.0000],
        [0.1839, 0.0514, 0.1499, 0.3818, 0.0289, 0.1437, 0.0603, 0.0000],
        [0.0533, 0.1844, 0.0814, 0.0034, 0.0140, 0.0214, 0.5796, 0.0625]],
       grad_fn=<SelectBackward0>)
torch.Size([4, 8, 8])
tensor([[-2.0433e-01,  2.5407e-01,  5.9613e-01,  1.2759e+00, -2.8305e-01,
          7.6875e-01,  1.5049e-02, -6.1012e-01,  1.0102e+00, -4.7575e-01,
         -7.0992e-01, -2.3299e-01, -1.6077e+00,  9.3972e-01,  4.6763e-02,
         -1.2855e-02],
        [-2.0186e-01,  5.4153e-02,  1.5977e-02,  9.8759e-01, -2.9155e-01,
          4.3639e-01,  3.3963e-0

In [ ]:
n_embd

64

In [143]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    # print(k)
    wei = q @ k.transpose(-2,-1) * C**-0.5
    # print(wei.shape)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)

    v = self.value(x)

    out = wei @ v

    return out


In [144]:
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.sa_head = Head(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None ):
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.sa_head(x)
    logits = self.lm_head(x)


    if targets is None:
      loss = None
    else:
      # print("targets \n",targets)
      B,T,C = logits.shape
      # print("logits \n",logits.shape)
      logits = logits.view(B*T,C)
      # print("logits reshape \n", logits.shape)
      targets = targets.view(B*T)
      # print("targets reshape \n", targets.shape)
      loss = F.cross_entropy(logits, targets)
      # print("loss \n",loss)

    return logits , loss


  def generate(self, idx, max_new_tokens):
    for k in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits, loss = self(idx_cond)
      # if k == 0 :
      #   print(logits.shape)
      logits= logits[:,-1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)

    return idx



In [145]:
idx = torch.zeros((1,1), dtype=torch.long , device=device)
model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(xb, yb)

# print(logits.shape)
# print(loss)
# print(loss.item())


print(decode(m.generate(idx=idx, max_new_tokens = 2)[0].tolist()))


OI


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for iter in range(max_iters):
      # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batches("train")
  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())


step 0: train loss 4.1604, val loss 4.1592
step 100: train loss 3.0700, val loss 3.0790
step 200: train loss 2.8853, val loss 2.8858
step 300: train loss 2.7945, val loss 2.8054
step 400: train loss 2.7451, val loss 2.7495
step 500: train loss 2.7218, val loss 2.7288
step 600: train loss 2.6936, val loss 2.6990
step 700: train loss 2.6826, val loss 2.6786
step 800: train loss 2.6523, val loss 2.6694
step 900: train loss 2.6418, val loss 2.6481
step 1000: train loss 2.6226, val loss 2.6292
step 1100: train loss 2.5974, val loss 2.6120
step 1200: train loss 2.5796, val loss 2.5781
step 1300: train loss 2.5577, val loss 2.5677
step 1400: train loss 2.5265, val loss 2.5427
step 1500: train loss 2.4906, val loss 2.5050
step 1600: train loss 2.4654, val loss 2.4656
step 1700: train loss 2.4360, val loss 2.4491
step 1800: train loss 2.4255, val loss 2.4362
step 1900: train loss 2.4141, val loss 2.4338
step 2000: train loss 2.4000, val loss 2.4176
step 2100: train loss 2.3903, val loss 2.4095


In [ ]:
idx = torch.zeros((2,2), dtype=torch.long , device=device)
print(decode(m.generate(idx=idx, max_new_tokens = 100)[0].tolist()))
print(decode(m.generate(idx=idx, max_new_tokens = 100)[1].tolist()))



Wewealif-
Wamyouoruss at-
chat yo wre gsakee. Sharde; me.
CETHO:
Why omallteaterel wo yary meepes fi


'tlo, nsu idotof st-th ther,
Thu is ond tuth yo led lothe witlal arel wiake acing thy to.

Frolo.

L


In [157]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    # print(q)
    wei = q @ k.transpose(-2,-1) * C**-0.5
    # print(wei)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)

    v = self.value(x)
    out = wei @ v
    return out

class MultiheadAttention(nn.Module):
  def __init__(self,num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd,n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward (self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out


In [158]:
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.sa_heads = MultiheadAttention(num_heads=num_heads,head_size=int(n_embd/num_heads) )
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None ):
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.sa_heads(x)
    logits = self.lm_head(x)


    if targets is None:
      loss = None
    else:
      # print("targets \n",targets)
      B,T,C = logits.shape
      # print("logits \n",logits.shape)
      logits = logits.view(B*T,C)
      # print("logits reshape \n", logits.shape)
      targets = targets.view(B*T)
      # print("targets reshape \n", targets.shape)
      loss = F.cross_entropy(logits, targets)
      # print("loss \n",loss)

    return logits , loss


  def generate(self, idx, max_new_tokens):
    for k in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits, loss = self(idx_cond)
      # if k == 0 :
      #   print(logits.shape)
      logits= logits[:,-1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)

    return idx



In [162]:
idx = torch.zeros((1,1), dtype=torch.long , device=device)
model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(xb, yb)

print(logits.shape)
print(loss)
print(loss.item())


print(decode(m.generate(idx=idx, max_new_tokens = 100)[0].tolist()))

torch.Size([512, 65])
tensor(4.2078, grad_fn=<NllLossBackward0>)
4.207807540893555

a3E$fSsISf:lTPZbpOlP-uap.doVgifTenCiyKuJLWHHbwd:$duddGHHoE.xdfuOWdBAzcMjttbeZNc$Nos,bvWX,d$jd-!pB:Y'


In [163]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [164]:
for iter in range(max_iters):
      # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batches("train")
  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())


step 0: train loss 4.1996, val loss 4.1988
step 100: train loss 2.9423, val loss 2.9562
step 200: train loss 2.7445, val loss 2.7376
step 300: train loss 2.6704, val loss 2.6678
step 400: train loss 2.6191, val loss 2.6153
step 500: train loss 2.5927, val loss 2.5805
step 600: train loss 2.5545, val loss 2.5423
step 700: train loss 2.5246, val loss 2.5237
step 800: train loss 2.4962, val loss 2.4903
step 900: train loss 2.4708, val loss 2.4576
step 1000: train loss 2.4401, val loss 2.4358
step 1100: train loss 2.4153, val loss 2.4145
step 1200: train loss 2.3930, val loss 2.3988
step 1300: train loss 2.3862, val loss 2.3904
step 1400: train loss 2.3503, val loss 2.3607
step 1500: train loss 2.3376, val loss 2.3537
step 1600: train loss 2.3316, val loss 2.3485
step 1700: train loss 2.3140, val loss 2.3316
step 1800: train loss 2.2997, val loss 2.3139
step 1900: train loss 2.2836, val loss 2.3049
step 2000: train loss 2.2748, val loss 2.2927
step 2100: train loss 2.2741, val loss 2.2914


In [165]:
print(decode(m.generate(idx=idx, max_new_tokens = 100)[0].tolist()))


her shave ker; are kes brik pon,
Dwitetl nga dith me gred mobeck.

QUEENE:
Wi's pre, and that feer'w


In [29]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    # print(q)
    wei = q @ k.transpose(-2,-1) * C**-0.5
    # print(wei)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)

    v = self.value(x)
    out = wei @ v
    return out

class MultiheadAttention(nn.Module):
  def __init__(self,num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd,n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward (self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out

class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, n_embd),
        nn.ReLU(),
    )

  def forward(self, x):
    return self.net(x)



class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.sa_heads = MultiheadAttention(num_heads=num_heads,head_size=int(n_embd/num_heads) )
    self.feedforward = FeedForward(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None ):
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.sa_heads(x)
    x = self.feedforward(x)
    logits = self.lm_head(x)


    if targets is None:
      loss = None
    else:
      # print("targets \n",targets)
      B,T,C = logits.shape
      # print("logits \n",logits.shape)
      logits = logits.view(B*T,C)
      # print("logits reshape \n", logits.shape)
      targets = targets.view(B*T)
      # print("targets reshape \n", targets.shape)
      loss = F.cross_entropy(logits, targets)
      # print("loss \n",loss)

    return logits , loss


  def generate(self, idx, max_new_tokens):
    for k in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits, loss = self(idx_cond)
      # if k == 0 :
      #   print(logits.shape)
      logits= logits[:,-1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)

    return idx



In [32]:
idx = torch.zeros((1,1), dtype=torch.long , device=device)
model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(xb, yb)

print(logits.shape)
print(loss)
print(loss.item())


print(decode(m.generate(idx=idx, max_new_tokens = 100)[0].tolist()))

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
      # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
      losses = estimate_loss()
      print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batches("train")
  logits, loss = m(xb, yb)

  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())


torch.Size([512, 65])
tensor(4.1557, grad_fn=<NllLossBackward0>)
4.155739784240723

La zM'VSGG q-lXmmqhesXK$Lyi&PGHiWQFkPnQe;NrUWCIYDcRlKhEVHmIH3&xGAKVQfSFe$SJTOnNiH J,
CoiEr
yfzH3XKik
step 0: train loss 4.1576, val loss 4.1558
step 100: train loss 2.9472, val loss 2.9678
step 200: train loss 2.6895, val loss 2.6957
step 300: train loss 2.6149, val loss 2.6165
step 400: train loss 2.5688, val loss 2.5715
step 500: train loss 2.5335, val loss 2.5340
step 600: train loss 2.4837, val loss 2.4782
step 700: train loss 2.4472, val loss 2.4437
step 800: train loss 2.4262, val loss 2.4155
step 900: train loss 2.4078, val loss 2.3903
step 1000: train loss 2.3779, val loss 2.3673
step 1100: train loss 2.3549, val loss 2.3620
step 1200: train loss 2.3337, val loss 2.3387
step 1300: train loss 2.3113, val loss 2.3074
step 1400: train loss 2.2981, val loss 2.2924
step 1500: train loss 2.2780, val loss 2.2920
step 1600: train loss 2.2634, val loss 2.2750
step 1700: train loss 2.2535, val loss 2.2613

In [33]:
print(decode(m.generate(idx=idx, max_new_tokens = 100)[0].tolist()))


And the sim he balown as lispakeorden piceed of
pengaing sight have,
Wike lord fajell if't uss me pa


In [ ]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    # print(q)
    wei = q @ k.transpose(-2,-1) * C**-0.5
    # print(wei)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)

    v = self.value(x)
    out = wei @ v
    return out

class MultiheadAttention(nn.Module):
  def __init__(self,num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd,n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward (self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out

class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout)

    )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):
  def __init__(self,n_embd,n_head):
    super().__init__()
    head_size = n_embd//num_heads
    self.sa = MultiheadAttention(n_head,head_size)
    self.feedforward = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.feedforward(self.ln2(x))
    return x



class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range (n_layer)])
    self.lnf = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None ):
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.lnf(x)
    x = self.feedforward(x)
    logits = self.lm_head(x)


    if targets is None:
      loss = None
    else:
      # print("targets \n",targets)
      B,T,C = logits.shape
      # print("logits \n",logits.shape)
      logits = logits.view(B*T,C)
      # print("logits reshape \n", logits.shape)
      targets = targets.view(B*T)
      # print("targets reshape \n", targets.shape)
      loss = F.cross_entropy(logits, targets)
      # print("loss \n",loss)

    return logits , loss


  def generate(self, idx, max_new_tokens):
    for k in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits, loss = self(idx_cond)
      # if k == 0 :
      #   print(logits.shape)
      logits= logits[:,-1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)

    return idx

